In [17]:
import openai
import os
from typing import List, Dict, Union
import json
from datasets import load_dataset
from tqdm import tqdm
from dotenv import load_dotenv
import json

In [5]:
ds = load_dataset("ai4privacy/pii-masking-300k", split="train[:100]")


Generating validation split: 100%|██████████| 47728/47728 [00:00<00:00, 284486.08 examples/s]


In [6]:
ds 


Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 100
})

In [15]:
ds['source_text'][0]


'Subject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- wynqvrh053 - Meeting at 10:20am\n- luka.burg - Meeting at 21\n- qahil.wittauer - Meeting at quarter past 13\n- gholamhossein.ruschke - Meeting at 9:47 PM\n- pdmjrsyoz1460 '

In [16]:
ds['target_text'][0]


'Subject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] '

In [26]:
with open('pii_redaction_results.json', 'r', encoding='utf-8') as f:
    redaction_results = json.load(f)


In [28]:
source = redaction_results[0]['original_source_text']
target = redaction_results[0]['original_target_text']
redacted = redaction_results[0]['redacted_text']

# for mask in redaction_results[0]['pii_entities']:
#     print(mask)
#     print(source[mask['start']:mask['end']])
#     print(target[mask['start']:mask['end']])
#     print(redacted[mask['start']:mask['end']])
#     print("--------------------------------")
print(redaction_results[0]['pii_entities'])
print(redaction_results[0]['original_privacy_mask'])

[{'type': 'USERNAME', 'value': 'wynqvrh053', 'start': 287, 'end': 297}, {'type': 'USERNAME', 'value': 'luka.burg', 'start': 321, 'end': 330}, {'type': 'USERNAME', 'value': 'qahil.wittauer', 'start': 349, 'end': 363}, {'type': 'USERNAME', 'value': 'gholamhossein.ruschke', 'start': 395, 'end': 416}, {'type': 'USERNAME', 'value': 'pdmjrsyoz1460', 'start': 440, 'end': 453}]
[{'value': 'wynqvrh053', 'start': 287, 'end': 297, 'label': 'USERNAME'}, {'value': '10:20am', 'start': 311, 'end': 318, 'label': 'TIME'}, {'value': 'luka.burg', 'start': 321, 'end': 330, 'label': 'USERNAME'}, {'value': '21', 'start': 344, 'end': 346, 'label': 'TIME'}, {'value': 'qahil.wittauer', 'start': 349, 'end': 363, 'label': 'USERNAME'}, {'value': 'quarter past 13', 'start': 377, 'end': 392, 'label': 'TIME'}, {'value': 'gholamhossein.ruschke', 'start': 395, 'end': 416, 'label': 'USERNAME'}, {'value': '9:47 PM', 'start': 430, 'end': 437, 'label': 'TIME'}, {'value': 'pdmjrsyoz1460', 'start': 440, 'end': 453, 'label':